In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Zephyr 7B


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer


In [ ]:
torch.set_default_device('cuda')

In [ ]:
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta",
                                             torch_dtype="auto",
                                             load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta",
                                          torch_dtype="auto",
                                          load_in_4bit=True)
streamer = TextStreamer(tokenizer)

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
text = """<s>[INST] What is your favourite condiment? [/INST]
        Well, I'm quite partial to a good squeeze of fresh lemon juice.
        It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! </s>
        <s>[INST] Do you have mayonnaise recipes? [/INST]
        """

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = torch.device("cuda")
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, streamer = streamer, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>  [INST] Do you have mayonnaise recipes? [/INST] Mayonnaise is a classic French condiment made by blending egg yolks with oil, vinegar, and other seasonings. Here's a simple recipe that I like to use:

Ingredients:

* 2 egg yolks
* 1 cup vegetable oil
* 1/4 cup red wine vinegar or lemon juice
* 1 tablespoon Dijon mustard
* Salt and pepper, to taste

Instructions:
1. In a small bowl, whisk together the egg yolks until well combined.
2. Slowly pour in the oil, whisking constantly, until the mixture becomes thick and creamy.
3. Add in the vinegar and mustard, and stir until combined.
4. Add salt and pepper to taste.
5. If you prefer a lighter mayonnaise, you can thin it out a bit by adding a few ice cubes and blending it briefly in a food processor or blender.
6. Store the mayonnaise

### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [ ]:
def generate(input_text, temperature=0.1, do_sample=True, max_new_tokens=100):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    model_inputs = inputs.to(device)
    outputs = model.generate(**model_inputs,
                             max_new_tokens=max_new_tokens,
                             temperature=temperature,
                             do_sample=do_sample,
                             streamer=streamer)

    tokenized_output = tokenizer.batch_decode(outputs[:, model_inputs["input_ids"].shape[1]:])[0]
    return tokenized_output

### ZeroShot Prompt

In [ ]:
prompt = '''Classify the following sentence in Positive, Neutral or Negative. \
Return only the classification of the sentence provided below.

Text: I went to the beach yesterday and I had a great time!
Answer:'''


output = generate(prompt, temperature=0.0, do_sample=False, max_new_tokens=5)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Classify the following sentence in Positive, Neutral or Negative. Return only the classification of the sentence provided below.

Text: I went to the beach yesterday and I had a great time!
Answer: Positive</s>


In [ ]:
output

'Positive</s>'

### FewShot Prompt

In [ ]:
prompt = '''Classify the following sentence in Positive, Neutral or Negative. \
Return only the classification of the sentence provided below. Do not add any new sentence.

Here are some examples of the task you have to solve:

Text: I went to the beach yesterday and I had a great time!
Answer: Positive

Text: I went to the mountain and it rained all day! We got home completely wet
Answer: Negative

Text: I went to the restaurant last night and the food was ok
Answer:'''


output = generate(prompt, temperature=0.0, do_sample=False, max_new_tokens=5)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Classify the following sentence in Positive, Neutral or Negative. Return only the classification of the sentence provided below. Do not add any new sentence.

Here are some examples of the task you have to solve:

Text: I went to the beach yesterday and I had a great time!
Answer: Positive

Text: I went to the mountain and it rained all day! We got home completely wet
Answer: Negative

Text: I went to the restaurant last night and the food was ok
Answer: Neutral




In [ ]:
output

'Neutral\n\n'

### Chain of Thought prompt

In [ ]:
prompt = """I went to the market and bought 10 apples.
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1.
How many apples did I remain with?
"""
output = generate(prompt, temperature=0.0, do_sample=False, max_new_tokens=256)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman. 
I then went and bought 5 more apples and ate 1. 
How many apples did I remain with?

Answer: 8 apples</s>


In [ ]:
output

'\nI remained with 8 apples.</s>'

In [ ]:
prompt = """I went to the market and bought 10 apples.
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1.
How many apples did I remain with?

Let's think step by step carefully and compute the solution one step at a time.
"""

output = generate(prompt, temperature=0.0, do_sample=False, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman. 
I then went and bought 5 more apples and ate 1. 
How many apples did I remain with?

Let's think step by step carefully and compute the solution one step at a time.

1. I started with 10 apples.
2. I gave away 2 apples to the neighbor and 2 to the repairman. That means I have 10 - 2 - 2 = 6 apples left.
3. Then I bought 5 more apples, which added to the remaining 6 apples. So now I have 6 + 5 = 11 apples.
4. Finally, I ate 1 apple. That means I have 11 - 1 = 10 apples left.

So, I remained with 10 apples.</s>


### Self Consistency prompt

In [ ]:
prompt = """Solve this math operation: (3+5)×4−7"""

In [ ]:
output1 = generate(prompt, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


old. When I was 6 my sister was half my age. Now
I’m 70 how old is my sister?

Let's think step by step carefully and compute the solution one step at a time.

1. When you were 6, your sister was half your age.
2. That means your sister was 6/2 = 3 years old.
3. Now you are 70 years old.
4. To find out how old your sister is, we subtract her age from yours: 70 - 3 = 67.

So, your sister is 67 years old.</s>


In [ ]:
output2 = generate(prompt, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


When I was 6 my sister was half my age. Now
I’m 70 how old is my sister?

Let's think step by step carefully and compute the solution one step at a time.

1. We know that you were 6 years old when your sister was half your age.
2. That means your sister was 3 years old.
3. Now, you are 70 years old.
4. To find out how old your sister is, we subtract her age from yours: 70 - 3 = 67.

So, your sister is currently 67 years old.</s>


In [ ]:
output3 = generate(prompt, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


When I was 6 my sister was half my age. Now
I’m 70 how old is my sister?

Let's think step by step carefully and compute the solution one step at a time.

1. We know that you were 6 years old when your sister was half your age.
2. That means your sister was 3 years old.
3. Now, you are 70 years old.
4. To find out how old your sister is, we subtract her age from yours: 70 - 3 = 67.

So, your sister is 67 years old.</s>


In [ ]:
print(output1)
print(output2)
print(output3)


1. When you were 6, your sister was half your age.
2. That means your sister was 6/2 = 3 years old.
3. Now you are 70 years old.
4. To find out how old your sister is, we subtract her age from yours: 70 - 3 = 67.

So, your sister is 67 years old.</s>

1. We know that you were 6 years old when your sister was half your age.
2. That means your sister was 3 years old.
3. Now, you are 70 years old.
4. To find out how old your sister is, we subtract her age from yours: 70 - 3 = 67.

So, your sister is currently 67 years old.</s>

1. We know that you were 6 years old when your sister was half your age.
2. That means your sister was 3 years old.
3. Now, you are 70 years old.
4. To find out how old your sister is, we subtract her age from yours: 70 - 3 = 67.

So, your sister is 67 years old.</s>
